In [1]:
import torch
import torch.nn as nn
import pandas as pd

In [2]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.constants import POSITION_COLS, MASS_CLASS_COLS, BASIC_TRAINING_COLS, FORCE_CLASS_COLS, PUCK_SQUARE_DISTANCES, PUCK_ANGLE_FEATURES
from isaac.sanity import class_proportions
from isaac.models import MultiBranchModel, initialise_model
from isaac.training import training_loop
import matplotlib.pyplot as plt
import numpy as np

In [3]:
data_directory = "seq_length_and_res_plots/"
model_directory = "models/seq_length_and_res/"

In [4]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

create_directory(data_directory)
create_directory(model_directory)

cuda:0


In [5]:
NORMALISE_DATA = True
BATCH_SIZE = 128
EPOCHS = 100
SEQ_END = 2700

In [6]:
base_resolution = 60 #fps
step_sizes = [int(base_resolution/resolution) for resolution in [20, 30, 60]]

base_length = 45 #s
seq_starts = [(base_length - length_in_s)*base_resolution for length_in_s in [45, 30, 15]]

In [7]:
seq_starts, step_sizes

([0, 900, 1800], [3, 2, 1])

In [8]:
train_trials = read_dataset("data/train_passive_trials.h5", n_trials=3500, cols=BASIC_TRAINING_COLS)
val_trials = read_dataset("data/val_passive_trials.h5", n_trials=1000, cols=BASIC_TRAINING_COLS)

100%|██████████| 1000/1000 [00:08<00:00, 118.21it/s]


# TRAINING

In [9]:
INPUT_DIM = len(BASIC_TRAINING_COLS)    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

network_params = (INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

In [10]:
loaders, scaler = prepare_dataset([train_trials, val_trials], 
                                  class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                  training_columns=BASIC_TRAINING_COLS, batch_size=BATCH_SIZE, 
                                  normalise_data=NORMALISE_DATA, device=device, multiclass=True)


stats_dfs = []

for step_size in step_sizes:
    for seq_start in seq_starts:
        print("step", step_size, "seq_start", seq_start)
        for seed in [0, 1, 2]:
            df = pd.DataFrame(columns=["step_size", "seq_start", "Epoch", "Mass Loss", "Force Loss", "Mass Train Accuracy",
                                       "Mass Val Accuracy", "Force Train Accuracy", "Force Val Accuracy", "seed"])

            model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device, arch=MultiBranchModel)
            epoch_losses, epoch_accuracies, [best_mass_model, best_force_model] = training_loop(model, optimizer, 
                                                                                                error, loaders[0], loaders[1], 
                                                                                                EPOCHS, seq_end=SEQ_END,
                                                                                                step_size=step_size,
                                                                                                seq_start=seq_start,
                                                                                                multibranch=True)

            torch.save(best_mass_model.state_dict(), "models/seq_length_and_res/best_mass_model_step_%d_start_%d_seed_%d.pt" % (step_size, seq_start, seed))
            torch.save(best_force_model.state_dict(), "models/seq_length_and_res/best_force_model_step_%d_start_%d_seed_%d.pt" % (step_size, seq_start, seed))

            train_accuracies = np.array(epoch_accuracies[0])
            val_accuracies = np.array(epoch_accuracies[1]) 

            df["Epoch"] = np.arange(EPOCHS)
            df["Mass Loss"] = epoch_losses[:, 0]
            df["Force Loss"] = epoch_losses[:, 1]        
            df["Mass Train Accuracy"] = train_accuracies[:, 0]
            df["Mass Val Accuracy"] = val_accuracies[:, 0]
            df["Force Train Accuracy"] = train_accuracies[:, 1]
            df["Force Val Accuracy"] = val_accuracies[:,1]
            df["step_size"] = step_size
            df["seq_start"] = seq_start            
            df["seed"] = str(seed)
            stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(data_directory+"stats.h5", key="stats")

  0%|          | 0/100 [00:00<?, ?it/s]

step 3 seq_start 0


Train_loss: ([0.8356112  0.89984073]) Train_acc: ([57.37142857 48.94285714]) Val_acc: ([59.  48.7]): 100%|██████████| 100/100 [20:39<00:00, 12.39s/it]
Train_loss: ([0.59038335 0.65417118]) Train_acc: ([67.14285714 59.71428571]) Val_acc: ([56.5 55.1]): 100%|██████████| 100/100 [20:40<00:00, 12.38s/it]
Train_loss: ([0.50793313 0.55336084]) Train_acc: ([76.02857143 69.68571429]) Val_acc: ([58.8 58.6]): 100%|██████████| 100/100 [20:38<00:00, 12.38s/it]
  0%|          | 0/100 [00:00<?, ?it/s]

step 3 seq_start 900


Train_loss: ([0.74230181 0.8201148 ]) Train_acc: ([59.34285714 54.22857143]) Val_acc: ([47.7 52.2]): 100%|██████████| 100/100 [14:41<00:00,  8.82s/it]
Train_loss: ([0.68039275 0.67618257]) Train_acc: ([68.17142857 62.8       ]) Val_acc: ([53.7 58. ]): 100%|██████████| 100/100 [14:41<00:00,  8.80s/it]
Train_loss: ([0.5465945  0.42083031]) Train_acc: ([70.77142857 71.28571429]) Val_acc: ([47.4 57. ]): 100%|██████████| 100/100 [14:41<00:00,  8.86s/it]
  0%|          | 0/100 [00:00<?, ?it/s]

...

Train_loss: ([0.41346774 0.38150167]) Train_acc: ([71.08571429 68.91428571]) Val_acc: ([59.  58.6]): 100%|██████████| 100/100 [53:58<00:00, 32.39s/it]
Train_loss: ([0.68868088 0.80683916]) Train_acc: ([61.6        49.02857143]) Val_acc: ([54.8 47.8]): 100%|██████████| 100/100 [53:59<00:00, 32.40s/it]
Train_loss: ([0.59044098 0.63974564]) Train_acc: ([65.45714286 61.14285714]) Val_acc: ([51.6 52.7]): 100%|██████████| 100/100 [53:57<00:00, 32.39s/it]
  0%|          | 0/100 [00:00<?, ?it/s]

step 1 seq_start 900


Train_loss: ([0.73335474 0.58777341]) Train_acc: ([58.31428571 51.68571429]) Val_acc: ([48.7 47.8]): 100%|██████████| 100/100 [37:20<00:00, 22.39s/it]
Train_loss: ([0.99526806 0.94470082]) Train_acc: ([44.08571429 42.68571429]) Val_acc: ([40.5 41.6]): 100%|██████████| 100/100 [37:22<00:00, 22.44s/it]
Train_loss: ([0.75820499 0.68545242]) Train_acc: ([57.4        54.94285714]) Val_acc: ([45.1 45.6]): 100%|██████████| 100/100 [37:08<00:00, 22.25s/it]
  0%|          | 0/100 [00:00<?, ?it/s]

step 1 seq_start 1800


Train_loss: ([0.80781169 0.71447451]) Train_acc: ([56.48571429 58.68571429]) Val_acc: ([43.5 48.3]): 100%|██████████| 100/100 [20:42<00:00, 12.44s/it]
Train_loss: ([0.96858977 0.92721928]) Train_acc: ([49.97142857 51.31428571]) Val_acc: ([40.8 43.2]): 100%|██████████| 100/100 [20:42<00:00, 12.42s/it]
Train_loss: ([1.02244215 0.95399802]) Train_acc: ([40.57142857 45.91428571]) Val_acc: ([37.9 43.1]): 100%|██████████| 100/100 [20:42<00:00, 12.42s/it]
